### Change status of a Supplementary File (opf) collection

Example use case: an ExperimentSet has many opf collections, but only one of these needs to be released.

For an opf collection, this notebook changes status of:
* all Files in the collection
* all input and output WorkflowRuns
* HiglassViewConfig for each file
* HiglassViewConfig for the collection
* QualityMetricWorkflowrun

In [ ]:
# Find all collections of other_processed_files

from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time

my_auth = get_key('')

# write ExpSet (or Experiment) accession or id here
exp_set_acc = ''  # e.g. '4DNES2M5JIGV'

exp_set = ff_utils.get_metadata(exp_set_acc, key=my_auth)
print('{} opf collections found in {}:\n'.format(
    len(exp_set.get('other_processed_files', [])), exp_set['accession']))
for coll in exp_set.get('other_processed_files', []):
    print(coll['title'])

In [ ]:
# Find all items in a collection
# NOTE: this list might not be exhaustive! Only items explicitly declared are found.

# copy here the collection name, from the list printed above
my_coll_name = ''  # e.g. 'Compartments Signals - Preliminary Files'

items_to_change = []
for coll in exp_set.get('other_processed_files', []):
    if coll['title'] == my_coll_name:
        my_coll = coll

# Files
files = [f for f in my_coll['files']]
items_to_change.extend(files)

# HiglassViewConfig of the Exp/Set
if my_coll.get('higlass_view_config'):
    items_to_change.append(my_coll['higlass_view_config'])

for f in files:
    a_file = ff_utils.get_metadata(f['@id'], key=my_auth)

    # HiglassViewConfig of the File
    for static_content in a_file.get('static_content', []):
        if static_content['content'].get('filetype', '') == 'HiglassViewConfig':
            items_to_change.append(static_content['content'])

    # WorkflowRuns
    wfrs = []
    wfrs.extend([wfr for wfr in a_file.get('workflow_run_inputs') if a_file.get('workflow_run_inputs')])
    wfrs.extend([wfr for wfr in a_file.get('workflow_run_outputs') if a_file.get('workflow_run_outputs')])
    if wfrs:
        items_to_change.extend(wfrs)
    
    # QualityMetrics
    for wf in wfrs:
        wf_item = ff_utils.get_metadata(wf['@id'], key=my_auth)
        if wf_item.get('quality_metric'):
            items_to_change.append(wf_item['quality_metric'])

# remove duplicates
new_list = []
id_found = []
for i in items_to_change:
    if i['@id'] not in id_found:
        id_found.append(i['@id'])
        if i.get('status') is None:
            i = ff_utils.get_metadata(i['@id'], key=my_auth)
        new_list.append(i)

# print items and status
print(len(new_list), 'items found:\n')
for i in new_list:
    print(i['@id'], i.get('status'), sep='\t')

In [ ]:
# patch the status
status = 'released to project'  # WARNING: this overwrites any other status, including archived, restricted, etc.
action = False  # set to True to patch

count = 0
for item in new_list:
    if item['status'] != status:
        print(item['@id'])
        if action:
            res = ff_utils.patch_metadata({'status': status}, item['@id'], key=my_auth)
            if res['status'] != 'success':
                print(res)
            else:
                count += 1
print('patched {} items'.format(count))